In [1]:
from transformers import CLIPProcessor, CLIPModel

In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
from torch.utils.data import DataLoader

In [5]:
from torchvision import datasets, transforms

In [6]:
from tqdm import tqdm

In [15]:
from PIL import Image

In [8]:
import os
import json

In [44]:
import random

In [17]:
# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [39]:
directory = r"C:\Users\Jlngo\Deep Learning in Computer Visions\Project\processed_data"

In [40]:
# List to store data from all JSON files
all_data = []

In [41]:
# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):  # Ensure the file is a JSON file
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            data['match'] = 1
            #print(type(data))
            '''
            if isinstance(data,dict) and 'Items' in data:
                for item in data['Items']:
                    item['match'] = 1
            else:
                data['match'] = 1
            '''
            all_data.append(data)

In [42]:
#print("Data loaded from all JSON files:", all_data)
print("Number of files loaded:", len(all_data))
print("Sample data:", all_data[0] if all_data else "No data loaded")

Number of files loaded: 12418
Sample data: {'Outfit_Gender': 'female', 'Outfit_Occasion': 'Campus', 'Outfit_Style': 'Casual', 'Items': [{'Image': '10000_9712_31465287787.jpg', 'category': ['Top'], 'subcategory': ['jacket']}, {'Image': '10000_9717_31942910318.jpg', 'category': ['Pants'], 'subcategory': ['casual pants']}, {'Image': '10000_6916_30454716460.jpg', 'category': ['Shoes'], 'subcategory': ['platform shoes']}], 'match': 1}


In [45]:
mismatch_data = []

In [47]:
all_images = [] #storing all the images for swapping 
class_data = {} #dictionary to store data by class for swapping 

In [53]:
for sample in all_data:
    if isinstance(sample,dict) and 'Items' in sample:
        #gender = sample.get('Outfit_Gender', 'Unknown')
        #occasion = sample.get('Outfit_Occasion','Unknown')
        style = sample.get('Outfit_Style','Unknown')
        #print(gender,occasion,style)
        for item in sample['Items']:
            all_images.append((style,item['Image'],item['category'],item['subcategory'],sample))
        if style not in class_data:
            class_data[style] = []
            class_data[style].append(sample)

In [55]:
count = len(all_data)
count

12418

In [59]:
for _ in range(count):
    #choosing a random sample 
    i = random.randint(0,len(all_data)-1)
    sample = all_data[i]
    #get the original class and items 
    original_class = sample.get('Outfit_Style','Unknown')
    original_item = sample['Items']
    #randomly choose an item to swap 
    swap_i = random.randint(0, len(original_item) - 1)
    item_to_swap = original_item[swap_i]
    #randomly select another sample from a different class
    swap_class = random.choice([cls for cls in class_data.keys() if cls != original_class])
    swap_sample = random.choice(class_data[swap_class])
    #get a random item from the selected swap class 
    swap_item = random.choice(swap_sample['Items'])
    #create mismatch by swapping the images 
    new_sample = sample.copy()  # Make a copy of the sample to avoid modifying the original
    new_sample['Items'][swap_i]['Image'] = swap_item['Image']  # Swap the image
    # Add the "match" field to the mismatch sample
    new_sample['match'] = 0  # This is a mismatch
    # Add to mismatch_data
    mismatch_data.append(new_sample)


In [61]:
print(len(mismatch_data))

12418


In [62]:
print("Sample data:", mismatch_data[0] if mismatch_data else "No data loaded")

Sample data: {'Outfit_Gender': 'female', 'Outfit_Occasion': 'Workplace', 'Outfit_Style': 'Elegant', 'Items': [{'Image': '10235_6908_30771792170.jpg', 'category': ['Skirt'], 'subcategory': ['dress']}, {'Image': '8046_5257_28283100161.jpg', 'category': ['Bags'], 'subcategory': ['square bag']}, {'Image': '8046_6917_26250549871.jpg', 'category': ['Shoes'], 'subcategory': ['Sandals']}], 'match': 0}


In [63]:
all_data.extend(mismatch_data)

In [65]:
len(all_data)

24836

In [67]:
output_file_path = r"C:\Users\Jlngo\Deep Learning in Computer Visions\Project\dataset.json"

In [68]:
with open(output_file_path, 'w') as output_file:
    json.dump(all_data, output_file, indent=4)

In [69]:
print(f"Dataset successfully written to {output_file_path}")

Dataset successfully written to C:\Users\Jlngo\Deep Learning in Computer Visions\Project\dataset.json
